In [1]:
import os
import ast
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

# 禁止随机，结果可复现
random.seed(42)
np.random.seed(42)

## 超参数选择

In [2]:
time_ratio = [1/2, 3/4, 7/8, 1][1]

In [3]:
# Parameters
stage_id = 0
data_number = 200

In [4]:
stage = ['计算矩阵', '读取矩阵'][stage_id]
model = 'Deep-learning/Ours'
emo_index = ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'][0]


ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering'
Clustering_Method = 'DTW_KMeans'
Emotion_Data_PATH = f'{ROOT_PATH}/data/Emotion_Data/{model}'   # 情绪数据路径
Financial_Data_PATH = f'{ROOT_PATH}/data/Financial_Data' # 金融数据路径

print(f"Running with: emo_index={emo_index}, stage={stage}, model={model}, data_number={data_number}")

Running with: emo_index=上证综合情绪值, stage=计算矩阵, model=Deep-learning/Ours, data_number=200


## 数据准备

In [5]:
"""读取股吧个股的数据"""
all_data = []

file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

for file in tqdm(file_list, desc="读取文件"):
    file_path = os.path.join(Emotion_Data_PATH, file)
    try:
        df = pd.read_csv(file_path, usecols=['日期', '高维情绪变量'])  # 只读需要的列
    except Exception as e:
        print(f"读取失败 {file}: {e}")
        continue

    stock_code = os.path.splitext(file)[0]

    df['股票编号'] = stock_code
    all_data.append(df)

guba_data = pd.concat(all_data, ignore_index=True)

读取文件:   0%|          | 0/183 [00:00<?, ?it/s]

读取文件:   1%|          | 1/183 [00:00<01:05,  2.80it/s]

读取文件:   1%|          | 2/183 [00:00<00:37,  4.82it/s]

读取文件:   2%|▏         | 3/183 [00:00<00:41,  4.30it/s]

读取文件:   2%|▏         | 4/183 [00:01<01:30,  1.98it/s]

...

读取文件:  98%|█████████▊| 179/183 [01:07<00:01,  3.18it/s]

读取文件:  99%|█████████▉| 181/183 [01:07<00:00,  4.30it/s]

读取文件:  99%|█████████▉| 182/183 [01:07<00:00,  4.20it/s]

读取文件: 100%|██████████| 183/183 [01:08<00:00,  2.03it/s]

读取文件: 100%|██████████| 183/183 [01:08<00:00,  2.65it/s]

In [6]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2024-12-31


In [7]:
guba_data = guba_data.sort_values(by="日期").reset_index(drop=True)
idx_ratio = int(len(guba_data) * time_ratio)  
guba_data = guba_data.iloc[:idx_ratio]
guba_data

,日期,高维情绪变量,股票编号
0,2021-01-01,"[-0.2253921627998352, 0.3089153468608856, 0.38...",601919
1,2021-01-01,"[0.08484992384910583, 0.4280889928340912, 0.46...",601012
2,2021-01-01,"[-0.23067481815814972, 0.23074224591255188, 0....",601318
3,2021-01-01,"[-0.16127993166446686, 0.09594926238059998, 0....",000625
4,2021-01-01,"[-0.22539249062538147, 0.308915376663208, 0.38...",000725
...,...,...,...
371493,2023-07-24,"[0.062030091881752014, 0.3784942030906677, 0.7...",600519
371494,2023-07-24,"[-0.21734082698822021, 0.02616022527217865, 0....",600519
371495,2023-07-24,"[-0.3522942364215851, -0.04511669650673866, 0....",600519
371496,2023-07-24,"[-0.12498360127210617, 0.14161376655101776, 0....",600519


In [8]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2023-07-24


In [9]:
"""读取股票回报率的数据"""
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [10]:
# 进行左连接，guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = merged_data.dropna()
merged_data

,日期,高维情绪变量,股票编号,交易量,收益率变化
6439,2021-06-01,"[-0.10334751009941101, -0.02684004232287407, 0...",601166,118168969.0,-0.022068
6440,2021-06-01,"[-0.12155835330486298, 0.17990195751190186, 0....",002241,119791643.0,0.034403
6441,2021-06-01,"[-0.406835675239563, 0.10095615684986115, 0.43...",601857,128459929.0,0.019481
6442,2021-06-01,"[-0.025604138150811195, -0.01249769888818264, ...",300896,2932326.0,0.004877
6443,2021-06-01,"[0.006676249671727419, 0.3031323552131653, 0.2...",300896,2932326.0,0.004877
...,...,...,...,...,...
371493,2023-07-24,"[0.062030091881752014, 0.3784942030906677, 0.7...",600519,1971337.0,-0.000085
371494,2023-07-24,"[-0.21734082698822021, 0.02616022527217865, 0....",600519,1971337.0,-0.000085
371495,2023-07-24,"[-0.3522942364215851, -0.04511669650673866, 0....",600519,1971337.0,-0.000085
371496,2023-07-24,"[-0.12498360127210617, 0.14161376655101776, 0....",600519,1971337.0,-0.000085


## 数据预处理

In [11]:
# 对每个股票编号的数据进行标准化和按日期汇总
tqdm.pandas()

def process_data(df):
    df_processed = []

    # 确保高维情绪变量是 list 类型（如果还没转过）
    if isinstance(df['高维情绪变量'].iloc[0], str):
        df['高维情绪变量'] = df['高维情绪变量'].progress_apply(ast.literal_eval)

    # 对每个股票编号处理，加 tqdm 进度条
    for stock_code, stock_data in tqdm(df.groupby('股票编号'), desc="Processing stocks"):
        for date, group in stock_data.groupby('日期'):
            try:
                emotion_matrix = np.array(group['高维情绪变量'].tolist())  # (样本数, 维度)
                avg_emotion = emotion_matrix.mean(axis=0).round(3).tolist()

                summary_row = {
                    '股票编号': stock_code,
                    '日期': date,
                    '高维情绪变量': avg_emotion,
                    '交易量': group['交易量'].mean(),
                    '收益率变化': group['收益率变化'].mean()
                }
                df_processed.append(summary_row)
            except Exception as e:
                print(f"跳过股票{stock_code} 日期{date}：{e}")
                continue

    return pd.DataFrame(df_processed)
    
# 调用处理函数
final_data = process_data(merged_data)
final_data

  0%|          | 0/290563 [00:00<?, ?it/s]

  0%|          | 30/290563 [00:00<16:17, 297.16it/s]

  0%|          | 90/290563 [00:00<10:15, 471.64it/s]

  0%|          | 150/290563 [00:00<09:11, 526.21it/s]

  0%|          | 210/290563 [00:00<08:45, 552.27it/s]

...

Processing stocks:  97%|█████████▋| 177/182 [00:19<00:00, 12.70it/s]

Processing stocks:  98%|█████████▊| 179/182 [00:20<00:00, 11.99it/s]

Processing stocks: 100%|██████████| 182/182 [00:20<00:00, 12.82it/s]

Processing stocks: 100%|██████████| 182/182 [00:20<00:00,  8.99it/s]

,股票编号,日期,高维情绪变量,交易量,收益率变化
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957
...,...,...,...,...,...
55300,688981,2023-07-17,"[-0.233, 0.002, 0.233, 0.221, -0.044, 0.278, -...",16507870.0,-0.013951
55301,688981,2023-07-18,"[-0.233, -0.003, 0.23, 0.234, 0.021, 0.288, -0...",27880112.0,0.019069
55302,688981,2023-07-19,"[-0.242, 0.066, 0.384, 0.179, -0.103, 0.272, -...",16050293.0,-0.011066
55303,688981,2023-07-20,"[-0.224, 0.15, 0.286, 0.267, -0.11, 0.267, -0....",13065589.0,0.002848


## DTW 聚类

In [12]:
# 选择需要的列
dtw_df = final_data[['日期', '股票编号', '高维情绪变量', '收益率变化']]

# 按照股票编号和日期排序
dtw_df = dtw_df.sort_values(by=['股票编号', '日期'])

# 创建一个以股票编号为键，日期为时间序列的字典
stock_data = {}
if data_number > len(dtw_df['股票编号'].unique()):
    data_number = len(dtw_df['股票编号'].unique())
for stock in dtw_df['股票编号'].unique()[:data_number]:
    stock_data[stock] = dtw_df[dtw_df['股票编号'] == stock][['日期', '高维情绪变量', '收益率变化']].reset_index(drop=True)

In [13]:
if stage == '计算矩阵':
    from tqdm import tqdm
    from fastdtw import fastdtw
    from scipy.spatial.distance import euclidean
    
    # 假设stock_data已经准备好了，stock_ids包含所有股票编号
    stock_ids = list(stock_data.keys())
    dtw_distances = np.zeros((len(stock_ids), len(stock_ids)))
    
    # 使用 tqdm 包装外层循环以显示进度条
    for i, stock_i in tqdm(enumerate(stock_ids), total=len(stock_ids), desc="计算 DTW 距离", unit="股票"):
        for j, stock_j in enumerate(stock_ids):
            if i < j:
                # 构造 series_i（二维数组，每行是 [情绪1, ..., 情绪N, 收益率]）
                df_i = stock_data[stock_i]
                series_i = [
                    emotion_vec + [ret]
                    for emotion_vec, ret in zip(df_i['高维情绪变量'], df_i['收益率变化'])
                ]

                df_j = stock_data[stock_j]
                series_j = [
                    emotion_vec + [ret]
                    for emotion_vec, ret in zip(df_j['高维情绪变量'], df_j['收益率变化'])
                ]

                # 使用 fastdtw 计算 DTW 距离
                distance, _ = fastdtw(series_i, series_j, dist=euclidean)
                dtw_distances[i, j] = distance
                dtw_distances[j, i] = distance  # 对称填充

    # 保存结果
    save_dir = f'{ROOT_PATH}/{Clustering_Method}/个股分析/{data_number}/{model}/个股分析_高维情绪'
    os.makedirs(save_dir, exist_ok=True)
    np.save(f'{save_dir}/dtw_distances.npy', dtw_distances)
    print(f'DTW距离矩阵已保存至：{save_dir}')

计算 DTW 距离:   0%|          | 0/182 [00:00<?, ?股票/s]

计算 DTW 距离:   1%|          | 1/182 [00:17<53:58, 17.89s/股票]

计算 DTW 距离:   1%|          | 2/182 [00:26<38:08, 12.71s/股票]

计算 DTW 距离:   2%|▏         | 3/182 [00:45<45:39, 15.31s/股票]

计算 DTW 距离:   2%|▏         | 4/182 [00:55<39:33, 13.33s/股票]

...

计算 DTW 距离:  98%|█████████▊| 178/182 [20:42<00:01,  2.66股票/s]

计算 DTW 距离:  98%|█████████▊| 179/182 [20:42<00:00,  3.28股票/s]

计算 DTW 距离:  99%|█████████▉| 181/182 [20:42<00:00,  5.17股票/s]

计算 DTW 距离: 100%|██████████| 182/182 [20:42<00:00,  6.83s/股票]

DTW距离矩阵已保存至：/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering/DTW_KMeans/个股分析/182/Deep-learning/Ours/个股分析_高维情绪


In [14]:
# 读取 DTW 距离矩阵
stock_ids = list(stock_data.keys())
dtw_distances = np.load(f'{ROOT_PATH}/{Clustering_Method}/个股分析/{data_number}/{model}/个股分析_高维情绪/dtw_distances.npy')

In [15]:
from sklearn.cluster import KMeans

# KMeans 聚类
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(dtw_distances)

# 查看每个股票的聚类结果
stock_clusters = {stock_ids[i]: clusters[i] for i in range(len(stock_ids))}

# 创建一个 DataFrame 来保存股票编号与对应的聚类标签
stock_clusters_df = pd.DataFrame({
    '股票编号': stock_ids,
    '聚类标签': clusters
})

# 保存到 CSV
os.makedirs('output', exist_ok=True)
stock_clusters_df.to_csv(f'output/{time_ratio}.csv', index=False)

# 查看结果
stock_clusters_df


,股票编号,聚类标签
0,000002,2
1,000061,1
2,000063,2
3,000069,0
4,000100,2
...,...,...
177,603986,2
178,688005,1
179,688027,1
180,688029,1
